In [2]:
import os

In [3]:
%pwd

'c:\\Users\\Admin\\OneDrive\\Desktop\\Data Science Project\\Text-Summarizer\\research'

In [4]:
os.chdir("../")

In [5]:
%pwd

'c:\\Users\\Admin\\OneDrive\\Desktop\\Data Science Project\\Text-Summarizer'

In [6]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    data_dir: Path
    tokenizer_name: Path

In [7]:
from textSummarizer.constants import *
from textSummarizer.utils.common import read_yaml_file, create_directories  

In [10]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FLIE_PATH,
        params_filepath = PARAMS_FILE_PATH
        ): 
        
        self.config = read_yaml_file(config_filepath)
        self.params = read_yaml_file(params_filepath)
        
        create_directories([self.config.artifacts_root])
        
    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        
        create_directories([config.root_dir])
        
        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        )
        
        return data_transformation_config
        

In [12]:
import os
from textSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_dataset,load_from_disk

In [15]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(self.tokenizer_name)
        
        
    def convert_examples_to_features(self, example_batch):
        input_encoding = self.tokenizer(example_batch['dialogue'], max_length = 1024, truncation=True)
        
        with self.tokenizer.as_target_tokenizer():
            target_encoding = self.tokenizer(example_batch['summary'], max_length=128, truncation=True)
            
        return {
            'input_ids': input_encoding['input_ids'],
            'attention_mask': input_encoding['attention_mask'],
            'labels': target_encoding['input_ids'],
        }
        
    def convert(self, features):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched=True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir,"samsum_dataset"))
        

In [16]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    data_transformation.validate_all_files_exist()
except Exception as e:
    raise e

[2024-03-23 22:28:44,973: INFO: common:yaml file: config\config.yaml loaded successfully]
[2024-03-23 22:28:44,982: INFO: common:yaml file: params.yaml loaded successfully]
[2024-03-23 22:28:44,984: INFO: common:Created directory at: artifacts ]
[2024-03-23 22:28:44,987: INFO: common:Created directory at: artifacts/data_transformation ]


BoxKeyError: "'ConfigBox' object has no attribute 'tokenizer_name'"